In [ ]:
import os
from matplotlib import pyplot as plt
from landcover_classification import prepare_training_data, load_training_data, classification, plot_classified_image
from wcps_rasdaman import wcps_rasdaman
from geo_utils import stack_geotiff
import datetime
from osgeo import gdal
from owslib.wcs import WebCoverageService
from glob import glob

## Input dataset
As data input for this demo we will use inSAR coherence data from a 5-D data cube we imported in Rasdaman for the SInCohMap project. Specifically for the landcover classification demo only the 36 images with the shortest baseline will be used (the images along the red line in the below coherence matrix).

The training data is derived from a mixture of the LandcoverInformationSystem (LISS) from South Tyrol, Open Street Map data to refine the training data in urban areas and the High Resolution Copernicus Forest layer to further refine the forest class. It is stored as shapefile.


<img src="coherence_matrix_shortest_baseline.png">

In the below image you can see the 5-D datacube in Rasdamans webinterface datacube
<img src="sincohmap_cube.PNG">


The Sincohmap data cube has two time axis (master & slave date), something owslib (we usally use to derive metadata information from Rasdaman via WCS) can´t work with. 
Therefore the information about the available timesteps must be gatherd manually from the DescribeCoverage response XML using the lxml.etree module.

In [ ]:
def extract_timepositions_sincohmap(wcs, coverage_name):
    from owslib.etree import etree
    from owslib.util import datetime_from_iso
    def nsWCS2(tag):
        return '{http://www.opengis.net/wcs/2.0}' + tag
    def gmlgrid(tag):
        return '{http://www.opengis.net/gml/3.3/rgrid}' + tag

    tree = wcs.getDescribeCoverage(coverage_name)

    # Find the Grid Element where all axis information is stored
    gridelem = tree.find(nsWCS2('CoverageDescription/') + '{http://www.opengis.net/gml/3.2}domainSet/' + gmlgrid(
        'ReferenceableGridByVectors'))

    all_axes = gridelem.findall(gmlgrid('generalGridAxis/') + gmlgrid('GeneralGridAxis'))

    # First axis is master, second is slave, save as datetime objects to list timepostions_master/slave

    master_date = all_axes[0].find(gmlgrid('coefficients')).text.split(' ')
    timepositions_master = []
    for x in master_date:
        x = x.replace('"', '')
        t_date = datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
        timepositions_master.append(t_date)

    slave_date = all_axes[1].find(gmlgrid('coefficients')).text.split(' ')
    timepositions_slave = []
    for x in slave_date:
        x = x.replace('"', '')
        t_date = datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
        timepositions_slave.append(t_date)
    return timepositions_master, timepositions_slave

In [ ]:
#################
# Define inputs #
#################
prepare_train = 0

#      COH  days  coh_VV coh_VH   6d   12d     18d    24d     36d    48d    60d    72d    84d    96d   INT   int_VV  int_VH    
inp = [True, 6,   True,   True,  True, False, False, False,  False, False, False, False, False, False, False, False, False]

#Define timespan for Coherence
COH      = inp[0]
TIMESPAN = inp[1]
coh_VV   = inp[2]
coh_VH   = inp[3]
_6days   = inp[4]
_12days  = inp[5]
_18days  = inp[6]
_24days  = inp[7]
_36days  = inp[8]
_48days  = inp[9]
_60days  = inp[10]
_72days  = inp[11]
_84days  = inp[12]
_96days  = inp[13]

# Define bands to use in intensity
INT      = inp[14]
int_VV   = inp[15]
int_VH   = inp[16]

# Select WCS server and service version

wcs = WebCoverageService('http://saocompute.eurac.edu/sincohmap/rasdaman/ows?', version='2.0.1')
coh_coverage = 'SOUTH_TYROL_4x19_UTM'
#int_coverage = 'SOUTH_TYROL_WEST_ASC_GEO_GAMMA_1x5_UTM_RESAMPLE'

#Define year
YEAR = 2017

# Define bbox + coordinate system (epsg:32632)
ulx = 651990
uly = 5181010
lrx = 671990
lry = 5161010
bbox_epsg32632 = 'E('+str(ulx)+':'+str(lrx)+'), N('+str(lry)+':'+str(uly)+'),' #more digits -> N (y) , E (x)

# Build filename from input
filename = ''
if (coh_coverage[:5]=='SOUTH'): filename = 'ST_'
filename += str(YEAR) + '_'
if COH:
    filename+='coh_'
    if coh_VV: filename += 'VV_'
    if coh_VH: filename += 'VH_'
    if _6days: filename += '6_'
    if _12days: filename += '12_'
    if _18days: filename += '18_'
    if _24days: filename += '24_'
    if _36days: filename += '36_'
    if _48days: filename += '48_'
    if _60days: filename += '60_'
    if _72days: filename += '72_'
    if _84days: filename += '84_'
    if _96days: filename += '96_'
if INT:
    filename += 'int_'
    if int_VV: filename += 'VV'
    if int_VH: filename += 'VH'


filename += '.tif'
print('[*] FILENAME: ',filename)

# Define the output filename of the data stack
stack_name = './Output_Data/ST/Sources/' + filename
timepositions_master, timepositions_slave = extract_timepositions_sincohmap(wcs, coh_coverage)


############################################
# Prepare the training/validation datasets #
############################################

# Define the number of sampling points & classification level (1 OR 3)
nr_points = 10000
class_level = 'LEVEL_3'
sampling_strategy = 'equal'

# Path to training data
training_path = './Training_Data/ST/ST_LVL3_bordermask.tif'
#vector_path = './Training_Data/ST/SInCohMap_STyrol_Validation_RR_WGS84_H32.shp'
vector_path = ''
# Mask for steep terrain
mask_path = './Training_Data/ST/geo_RR_combined_mask_radar_inv.tif'
# path of where to store the extracted training samples
sample_path = './Training_Data/ST/ST_training_10000_samples_equal.p'

# define the path to where to store your resulting classified map
class_path = './Output_Data/ST/Classified/classified_' + filename


Now that we have the timepostions, we can acutally send WCPS requests to the Rasdaman server. This is done with the* wcps_rasdaman* module, which sends the request to the server and converts the response in a way that we can work with it directly in Python.
In this example we are downloading TIFFs of the single images (VH band) and stack it to a multiband TIFF.

In [ ]:
###################################################
# Download raster data subset via WCPS & stack it #
###################################################
print("TIMESPAN = ", TIMESPAN)
files = []
index = 0
for i in range(len(timepositions_master)):
    if (timepositions_master[i].year == YEAR):
        if index == 0:
            master_reference = timepositions_master[i]
            print("REF: ", master_reference)
            index = 1
        timestep_master = timepositions_master[i].isoformat()
        print("MASTER: ", timestep_master)
        if COH:
            for j in range(len(timepositions_slave)):
                if (timepositions_slave[j].year == YEAR) and \
                   (timepositions_slave[j].timetuple().tm_yday > timepositions_master[i].timetuple().tm_yday + 0) and \
                   (timepositions_slave[j].timetuple().tm_yday <= timepositions_master[i].timetuple().tm_yday + TIMESPAN):              
                    timestep_slave = timepositions_slave[j].isoformat()
                    print("SLAVE: ",timepositions_slave[j].isoformat())
                    if(i > 0):
                        if ((timepositions_master[i].timetuple().tm_yday-master_reference.timetuple().tm_yday) % TIMESPAN != 0):
                            continue
                    if not _6days:
                        if (timepositions_slave[j].timetuple().tm_yday - timepositions_master[i].timetuple().tm_yday) == 6:
                            continue
                    if not _12days:
                        if (timepositions_slave[j].timetuple().tm_yday - timepositions_master[i].timetuple().tm_yday) == 12:
                            continue
                    if not _18days:
                        if (timepositions_slave[j].timetuple().tm_yday - timepositions_master[i].timetuple().tm_yday) == 18:
                            continue
                    if not _24days:
                        if (timepositions_slave[j].timetuple().tm_yday - timepositions_master[i].timetuple().tm_yday) == 24:
                            continue
                    print("SLAVE MASTER DIFF: ",timepositions_slave[j].timetuple().tm_yday - timepositions_master[i].timetuple().tm_yday)
                    if coh_VV:
                        query_coh_VV = ('for c in (' + coh_coverage + ') return encode' + 
                                 '(((float) c.Coherence_VV[' + bbox_epsg32632 +
                                     'master_date("' + timestep_master + '"), ' + 
                                     'slave_date("' + timestep_slave + '")' + 
                                 ']),"tiff")')
                        subset_coherence_VV = wcps_rasdaman(query_coh_VV, ip = 'saocompute.eurac.edu/sincohmap')
                        files.append(subset_coherence_VV)
                    if coh_VH:
                        query_coh_VH = ('for c in (' + coh_coverage + ') return encode' + 
                                 '(((float) c.Coherence_VH[' + bbox_epsg32632 +
                                     'master_date("' + timestep_master + '"), ' + 
                                     'slave_date("' + timestep_slave + '")' + 
                                 ']),"tiff")')
                        subset_coherence_VH = wcps_rasdaman(query_coh_VH, ip = 'saocompute.eurac.edu/sincohmap')
                        files.append(subset_coherence_VH)
        if INT:
            if int_VV:
                query_int_VV = ('for c in (' + int_coverage + ') return encode' + 
                     '(((float) 10*log(c.Intensity_VV[' + bbox_epsg32632 +
                         'date("' + timestep_master + '") ' + 
                     '])),"tiff")')
                subset_int_VV = wcps_rasdaman(query_int_VV, ip = 'saocompute.eurac.edu/sincohmap')
                files.append(subset_int_VV)
            if int_VH:
                query_int_VH = ('for c in (' + int_coverage + ') return encode' + 
                         '(((float) 10*log(c.Intensity_VH[' + bbox_epsg32632 +
                             'date("' + timestep_master + '") ' + 
                         '])),"tiff")')
                subset_int_VH = wcps_rasdaman(query_int_VH, ip = 'saocompute.eurac.edu/sincohmap')
                files.append(subset_int_VH)

stack_geotiff(files, outtif=stack_name) #the raster image to be classififed with scikit-learn


## Landcover classification for CORINE Level 3 with Random Forests

In [ ]:
# ################################
# # Print data stack information #
# ################################

raster_dataset = gdal.Open(stack_name, gdal.GA_ReadOnly)
geo_transform = raster_dataset.GetGeoTransform()
print(geo_transform)
extent = []
extent.append(geo_transform[0])
extent.append(geo_transform[0] + geo_transform[1]*raster_dataset.RasterXSize)
extent.append(geo_transform[3] + geo_transform[5]*raster_dataset.RasterYSize)
extent.append(geo_transform[3])
print(extent)
print("X: ", raster_dataset.RasterXSize)
print("Y: ", raster_dataset.RasterYSize)

In [ ]:
###############################
# Create new training samples #
###############################

#training_samples, training_labels, bands_data, projection, geo_transform,training_pixels = prepare_training_data(training_path, stack_name, class_level, nr_points, sampling_strategy, mask_path=mask_path, sample_path=sample_path)

In [ ]:
###############################
# Load saved training samples #
###############################

# this step is necessary if you want to use the same training pixels locations generated previously and stored
training_samples, training_labels, bands_data, projection, geo_transform, training_pixels = load_training_data(stack_name, training_path, sample_path)

In [ ]:
################################
# Random forest classification #
################################

test_labels = [None]
test_samples = [None]
classified_image, classifier = classification(training_samples, training_labels, test_labels, test_samples, bands_data, projection, geo_transform,'rf', gridsearch=False, mask_path =mask_path, n_estimators = 1000, class_path=class_path)

In [ ]:
###############################
# Enjoy the looks of your MAP #
###############################

# plot your image to see the resulting classified map
%matplotlib inline
plot_classified_image(class_path=class_path, plot_map_info=True, plot_map_legend=True, plot_title=class_path)